In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
data = pd.read_csv('all_stocks_5yr.csv')

In [22]:
data

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [11]:
stocks = set(data['Name'])

stock_dict = {}
for stock in stocks:
    stock_dict[stock] = []

In [21]:
for i, row in data.iterrows():
    stock_dict[row['Name']].append(dict(row))

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [17]:
stock_dict['AAL']

date      2018-02-07
open           50.91
high           51.98
low            50.89
close           51.4
volume       4845831
Name             AAL
Name: 1258, dtype: object